In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import joblib

import numpy as np
import pandas as pd
import plotly.express as px

from typing import List, Optional

from datetime import date, datetime, timedelta
from pytz import timezone

import optuna
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

import hopsworks
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [3]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [4]:
import src.config as config

In [5]:
# Connect to Hopsworks
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)

# Get the feature store handle for the project
feature_store = project.get_feature_store()

# Get the feature group
feature_group = feature_store.get_or_create_feature_group(name=config.FEATURE_GROUP_NAME, 
                                                          version=config.FEATURE_GROUP_VERSION,
                                                          description='Timeseries data with hourly frequency', 
                                                          primary_key=['pickup_location_id', 'pickup_ts'], 
                                                          event_time='pickup_hour')

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/699541
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Create the feature view
try:
    #
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print(f'Feature View already exists')

# Get the feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, 
    version=config.FEATURE_VIEW_VERSION
    )

Feature View already exists


In [7]:
ts_data, _ = feature_view.training_data(
    description='Timeseries data with hourly frequency',
)

2024-05-10 00:06:04,699 WARNING: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.64s) 
2024-05-10 00:06:23,899 WARNING: VersionWarning: Incremented version to `11`.



In [8]:
# Drop the pickup_ts column
ts_data.drop('pickup_ts', axis=1, inplace=True)

#
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
380997,2023-01-01 00:00:00+00:00,0,1
1958924,2023-01-01 01:00:00+00:00,0,1
2579941,2023-01-01 02:00:00+00:00,0,1
2599272,2023-01-01 03:00:00+00:00,0,1
2954848,2023-01-01 04:00:00+00:00,0,1
...,...,...,...
276766,2024-05-08 19:00:00+00:00,5,265
1870901,2024-05-08 20:00:00+00:00,5,265
1912279,2024-05-08 21:00:00+00:00,6,265
308546,2024-05-08 22:00:00+00:00,4,265


In [9]:
def plot_ts(
        ts_data: pd.DataFrame,
        locations: Optional[List[int]] = None
        ):
    
    '''
    Plot time-series data for the specified locations
    '''

    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x='pickup_hour',
        y='rides',
        color='pickup_location_id',
        template='none'
    )

    fig.show()

In [10]:
plot_ts(ts_data, locations=[265])

In [11]:
from src.data import transform_timeseries_data_into_features_targets

In [12]:
features, targets = transform_timeseries_data_into_features_targets(
    ts_data, 
    input_sequence_length=24*28, # 1 month
    step_size=23)

features_and_targets = features.copy()
features_and_targets['target_rides_next_hour'] = targets

100%|██████████| 265/265 [02:10<00:00,  2.03it/s]


In [13]:
print(f'{features_and_targets.shape=}')

features_and_targets.shape=(129055, 675)


In [14]:
features_and_targets

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29 00:00:00+00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,...,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2023-01-29 23:00:00+00:00,1,0.0
2,2.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,2.0,1.0,2.0,0.0,1.0,0.0,2023-01-30 22:00:00+00:00,1,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,2.0,0.0,0.0,0.0,2023-01-31 21:00:00+00:00,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,1.0,0.0,2.0,1.0,1.0,0.0,2023-02-01 20:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129050,1.0,7.0,3.0,5.0,10.0,4.0,6.0,1.0,1.0,2.0,...,1.0,3.0,2.0,4.0,2.0,1.0,4.0,2024-05-04 22:00:00+00:00,265,5.0
129051,3.0,7.0,6.0,4.0,1.0,4.0,0.0,1.0,2.0,2.0,...,1.0,3.0,3.0,3.0,5.0,3.0,1.0,2024-05-05 21:00:00+00:00,265,2.0
129052,2.0,1.0,2.0,3.0,2.0,6.0,0.0,0.0,1.0,0.0,...,0.0,14.0,4.0,3.0,7.0,4.0,5.0,2024-05-06 20:00:00+00:00,265,2.0
129053,1.0,1.0,1.0,1.0,2.0,4.0,3.0,1.0,1.0,0.0,...,7.0,6.0,4.0,12.0,9.0,10.0,11.0,2024-05-07 19:00:00+00:00,265,8.0


In [15]:
from src.data_split import train_test_split

In [16]:
cutoff_date = pd.to_datetime(datetime.utcnow() - timedelta(days=28)) 
cutoff_date = cutoff_date.tz_localize('UTC')
print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_targets, 
    cutoff_date,
    target_column_name='target_rides_next_hour'
)

cutoff_date=Timestamp('2024-04-12 05:08:39.467815+0000', tz='UTC')


In [17]:
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(121635, 674)
y_train.shape=(121635,)
X_test.shape=(7420, 674)
y_test.shape=(7420,)


In [18]:
from src.model import get_pipeline

In [19]:
import warnings

warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [20]:
def objective(trial: optuna.trial.Trial) -> float:

    '''
    Define the objective function to optimize the hyperparameters of the model
    Based on a time series split 
    '''

    # Define the hyperparameters to optimize
    hyperparameters = {
        'metric': 'mae',
        'verbose': -1,
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
    }

    # Sort the training data by pickup_hour
    # Allows TimeSeriesSplit to split the data in a consistent method
    X_train_sorted = X_train.sort_values(by='pickup_hour').reset_index(drop=True)
    y_train_sorted = y_train.reindex_like(X_train_sorted)

    tss = TimeSeriesSplit(n_splits=2)
    scores = []

    for train_index, val_index in tss.split(X_train):
        # Split the data into training and validation sets
        X_train_, X_val_ = X_train_sorted.iloc[train_index, :], X_train_sorted.iloc[val_index, :]
        y_train_, y_val_ = y_train_sorted.iloc[train_index], y_train_sorted.iloc[val_index]

        # Train the model
        pipeline = get_pipeline(**hyperparameters)
        pipeline.fit(X_train_, y_train_)

        # Evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)
        scores.append(mae)

    return np.array(scores).mean()

In [21]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-05-10 00:08:41,814] A new study created in memory with name: no-name-19a51735-89a8-4384-890c-1fa06034ddae


[LightGBM] [Warning] bagging_fraction is set=0.32723610232358685, subsample=1.0 will be ignored. Current value: bagging_fraction=0.32723610232358685
[LightGBM] [Warning] feature_fraction is set=0.7196048758428861, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7196048758428861
[LightGBM] [Warning] bagging_fraction is set=0.32723610232358685, subsample=1.0 will be ignored. Current value: bagging_fraction=0.32723610232358685
[LightGBM] [Warning] feature_fraction is set=0.7196048758428861, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7196048758428861


[I 2024-05-10 00:09:33,298] Trial 0 finished with value: 27.20853285726467 and parameters: {'num_leaves': 91, 'feature_fraction': 0.7196048758428861, 'bagging_fraction': 0.32723610232358685, 'min_child_samples': 37}. Best is trial 0 with value: 27.20853285726467.


[LightGBM] [Warning] bagging_fraction is set=0.4339502826014176, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4339502826014176
[LightGBM] [Warning] feature_fraction is set=0.2763185584273332, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2763185584273332
[LightGBM] [Warning] bagging_fraction is set=0.4339502826014176, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4339502826014176
[LightGBM] [Warning] feature_fraction is set=0.2763185584273332, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2763185584273332


[I 2024-05-10 00:09:55,590] Trial 1 finished with value: 27.369248357597023 and parameters: {'num_leaves': 33, 'feature_fraction': 0.2763185584273332, 'bagging_fraction': 0.4339502826014176, 'min_child_samples': 21}. Best is trial 0 with value: 27.20853285726467.


...

[I 2024-05-10 01:03:01,694] Trial 97 finished with value: 27.58535890974641 and parameters: {'num_leaves': 146, 'feature_fraction': 0.8490917847126868, 'bagging_fraction': 0.2882308099481158, 'min_child_samples': 49}. Best is trial 38 with value: 26.94863798303215.


[LightGBM] [Warning] bagging_fraction is set=0.2515517909908591, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2515517909908591
[LightGBM] [Warning] feature_fraction is set=0.791360609182423, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.791360609182423
[LightGBM] [Warning] bagging_fraction is set=0.2515517909908591, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2515517909908591
[LightGBM] [Warning] feature_fraction is set=0.791360609182423, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.791360609182423


[I 2024-05-10 01:03:32,708] Trial 98 finished with value: 27.111666832028142 and parameters: {'num_leaves': 36, 'feature_fraction': 0.791360609182423, 'bagging_fraction': 0.2515517909908591, 'min_child_samples': 46}. Best is trial 38 with value: 26.94863798303215.


[LightGBM] [Warning] bagging_fraction is set=0.57489238898797, subsample=1.0 will be ignored. Current value: bagging_fraction=0.57489238898797
[LightGBM] [Warning] feature_fraction is set=0.6015977560267526, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6015977560267526
[LightGBM] [Warning] bagging_fraction is set=0.57489238898797, subsample=1.0 will be ignored. Current value: bagging_fraction=0.57489238898797
[LightGBM] [Warning] feature_fraction is set=0.6015977560267526, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6015977560267526


[I 2024-05-10 01:03:57,536] Trial 99 finished with value: 27.112137280251105 and parameters: {'num_leaves': 13, 'feature_fraction': 0.6015977560267526, 'bagging_fraction': 0.57489238898797, 'min_child_samples': 19}. Best is trial 38 with value: 26.94863798303215.


In [22]:
best_parameters = study.best_trial.params
print(f'{best_parameters=}')

best_parameters={'num_leaves': 19, 'feature_fraction': 0.8608206393043076, 'bagging_fraction': 0.5153861087184175, 'min_child_samples': 90}


In [23]:
pipeline = get_pipeline(**best_parameters)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.5153861087184175, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5153861087184175
[LightGBM] [Warning] feature_fraction is set=0.8608206393043076, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8608206393043076


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x14dea7f70>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.5153861087184175,
                               feature_fraction=0.8608206393043076,
                               min_child_samples=90, num_leaves=19))])

In [24]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae:.2f}')

2.89


In [25]:
from src.paths import MODELS_DIR

In [26]:
joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/Users/cmartinez/Documents/08_projects/taxi_demand_prediction_solution/models/model.pkl']

In [27]:
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [28]:
model_registry = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [29]:
model = model_registry.sklearn.create_model(
    name='taxi_demand_prediction',
    metrics={'test_mae': test_mae},
    description='LightGBM regression model with hyperparameter tuning to predict the number of rides in the next hour',
    input_example=X_train.sample(),
    model_schema=model_schema
)

In [31]:
model.save(str(MODELS_DIR / 'model.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/239295 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/4461 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/58136 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/699541/models/taxi_demand_prediction/1


Model(name: 'taxi_demand_prediction', version: 1)